In [0]:
%pip install mlflow pydantic databricks-agents -U --quiet
%pip install unitycatalog-crewai unitycatalog-crewai[databricks] -U --quiet
%pip install crewai==0.102 crewai_tools langchain_community --quiet
%restart_python

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import mlflow
from mlflow.tracking import MlflowClient
from bs4.diagnose import profile
from dotenv import load_dotenv
from databricks.connect import DatabricksSession
import os

os.environ["DATABRICKS_API_KEY"] = dbutils.secrets.get("databricks_token_qyu", "qyu_rag_sp_token")
os.environ["DATABRICKS_API_BASE"] = (
    f"{dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()}/serving-endpoints"
)

mlflow.set_tracking_uri("databricks")
mlflow.set_registry_uri("databricks-uc")


experiment = "/Workspace/Users/q.yu@databricks.com/ML_experiments/insurance_operation_crewai_agent"
mlflow.set_experiment(experiment)

mlflow.crewai.autolog()

In [0]:
from unitycatalog.ai.core.base import set_uc_function_client
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()
set_uc_function_client(client)
CATALOG = "fins_genai"
SCHEMA = 'insurance_agent'

spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SCHEMA}")

In [0]:
%sql
select *
from fins_genai.insurance_agent.search_claims_details_by_policy_no("102070455")

In [0]:
%sql
select *
from fins_genai.insurance_agent.policy_doc_vector_search("car accident")

# Create a Crew

## Set tools

In [0]:
from unitycatalog.ai.crewai.toolkit import UCFunctionToolkit

uc_tools = [f"{CATALOG}.{SCHEMA}.{func.name}" for func in client.list_functions(catalog=CATALOG,
                                                                                schema=SCHEMA)
            if func.name in ['policy_doc_vector_search', 
                             'search_claims_details_by_policy_no']]
print(uc_tools)
toolkit = UCFunctionToolkit(function_names=uc_tools)
tools = toolkit.tools
search_policy_doc_tool, search_claim_details_tool = tools

In [0]:
from crewai import Agent, Task, Crew, Process
from crewai import LLM
from crewai.tools import BaseTool

llm = LLM(model="databricks/databricks-meta-llama-3-3-70b-instruct")
#llm = LLM(model="databricks/databricks-llama-4-maverick")
# llm = LLM(model="databricks/databricks-claude-3-7-sonnet")
llm = LLM(model="databricks/doan-gpt-4o")


## Create Agents

In [0]:
triage_agent = Agent(
    role="triage agent",
    goal="analysis customer query and delegate to appropriate specialist, if you cannot "
            "identify the goal of query, ask user for clarification",
    backstory="The triage agent is responsible for analyzing the customer query and "
                "delegate the task to the appropriate agent. If the goal query cannot be "
                "identified, ask the user for clarification",
    verbose=False,
    allow_delegation=True,
    llm=llm
)

claim_agent = Agent(
    role="claim agent",
    goal="Provide accurate and helpful information about customer policy and claims",
    backstory=""" You are an insurance claims specialist. 
        If you are speaking to a customer, you probably were transferred to you from the triage agent.
        If the customer does not have a policy number, explain that you need it to access their claim information.
        Your job is to:
        1. answer customer inquiries related to claims
        2. Look up customer profiles or retrieve claim information
        3. Provide information on existing claims
        """,
    verbose=False,
    allow_delegation=False,
    tools=[search_claim_details_tool],
    llm=llm
)

policy_agent = Agent(
    role="policy agent",
    goal="handle policy related queries",
    backstory="""You are a general policy FAQ agent.
              If you are speaking to a customer, you probably were transferred to you from the triage agent.
              Your job is to:
             1. Identify the last question asked by the customer. 
             2. Use the search tools to answer the question. Do not rely on your own knowledge.""",
    verbose=False,
    allow_delegation=False,
    tools=[search_policy_doc_tool],
    llm=llm
)

## Create Tasks

In [0]:
triage_task = Task(
    description="Analyze the customer query: {query} route to appropriate "
                "specialist",
    expected_output="return the word claim, policy, or unknown",
    agent=triage_agent,
)

claim_task = Task(
    description=(
        """Process this claim-related customer request based on: 
        query: {query}

        If you have a policy number,
        - use the search_claims_details_by_policy_no tool to retrieve existing claim information.
        
        Provide a helpful, detailed response about their claim.If you don't have their
        policy_number, explain that you need it to access their claim information."""
    ),
    expected_output="A helpful response to the customer's claim query",
    agent=claim_agent,
)

policy_task = Task(
    description="Answer the general policy related query: {query}",
    expected_output="answer the general policy queries use the policy_docs_vector_search tool retrieve the policy document info",
    agent=policy_agent
)

## Create a crew

In [0]:
crew = Crew(
  agents=[triage_agent, claim_agent, policy_agent],
  tasks=[triage_task, claim_task, policy_task],
  process=Process.hierarchical,
  manager_llm=llm,
  verbose=False
)

# Running a insurance operation crew

In [0]:
with mlflow.start_span(name="insurance_agent", span_type="AGENT") as span:
    print("[AGENT] Hello! How may I assist you?")
    while True:
        user_input = input("[USER]: ")
        if user_input.lower() == "exit":  # Break loop if user types 'exit'
            print("[AGENT]: Bye!")
            break
        if not user_input:
            continue
        try:
            result = crew.kickoff({"query": user_input})
            print("\n[AGENT]:", result.raw)
        except Exception as e:
            print(f"\nError occurred: {str(e)}")

# logging the crewAI workflow as ChatAgent